In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

In [9]:
final_data=pd.read_csv('data.csv')

In [10]:
# Select relevant columns
columns = ['Employee Estimate', 'Employees on Professional Networks', 'Employee Growth (Monthly)', 
           'Employee Growth (Quarterly)', 'Employee Growth (6 months)', 'Employee Growth (Annual)', 
           'Headquarters', 'Year Founded', 'Last Funding Amount', 'Last Funding Date', 
           'Total Funding Amount', 'Total Funding Rounds', 'Business Model', 'Last Touch Date', 
           'Last Pipeline Decline Date', 'Rank']
df = final_data[columns].copy()

# US location check
us_states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

def is_us_location(location):
    if pd.isna(location):
        return 0.5
    return int(any(state in location for state in us_states))

df['is_us'] = df['Headquarters'].apply(is_us_location)

# Calculate company age
current_year = datetime.now().year
df['company_age'] = current_year - df['Year Founded']

# Check for tech-enabled sectors
tech_enabled_sectors = ['Software', 'Software Enabled', 'Tech', 'Technology']

def is_tech_enabled(sector):
    if pd.isna(sector):
        return 0
    return int(sector in tech_enabled_sectors)

df['is_tech_enabled'] = df['Business Model'].apply(is_tech_enabled)

df['Rank'] -= 1

# Clean data
df_cleaned = df.drop(columns=['Year Founded', 'Headquarters', 'Business Model'])
df_cleaned = df_cleaned.dropna(subset=['Rank'])

# Convert 'Employee Growth' columns to float
growth_columns = ['Employee Growth (Monthly)', 'Employee Growth (Quarterly)', 'Employee Growth (6 months)', 'Employee Growth (Annual)']
for col in growth_columns:
    df_cleaned[col] = df_cleaned[col].str.replace('%', '').astype(float)

# Process date columns
today = pd.Timestamp.now().tz_localize(None)

def safe_to_timestamp(x):
    if pd.isna(x):
        return np.nan
    try:
        return pd.to_datetime(x).tz_localize(None)
    except:
        return np.nan

date_columns = ['Last Funding Date', 'Last Touch Date', 'Last Pipeline Decline Date']
for col in date_columns:
    df_cleaned[col] = df_cleaned[col].apply(safe_to_timestamp)
    df_cleaned[f'Days Since {col}'] = (today - df_cleaned[col]).dt.days.replace({pd.NaT: np.nan})

# Drop original date columns
df_cleaned = df_cleaned.drop(columns=date_columns)

# Save the cleaned data to a CSV file
df_cleaned.to_csv('train_data.csv', index=False)

In [11]:
df_cleaned

,Employee Estimate,Employees on Professional Networks,Employee Growth (Monthly),Employee Growth (Quarterly),Employee Growth (6 months),Employee Growth (Annual),Last Funding Amount,Total Funding Amount,Total Funding Rounds,Rank,is_us,company_age,is_tech_enabled,Days Since Last Funding Date,Days Since Last Touch Date,Days Since Last Pipeline Decline Date
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.5,5.0,0,NaN,NaN,NaN
1,81.0,81.0,3.85,1.25,2.53,28.57,14000000.0,29000000.0,4.0,7.0,1.0,8.0,1,730.0,323.0,NaN
2,21.0,22.0,0.00,0.00,-4.35,15.79,NaN,NaN,0.0,5.0,1.0,3.0,1,NaN,323.0,NaN
3,29.0,29.0,0.00,-3.33,-12.12,-9.38,8000000.0,8000000.0,2.0,6.0,0.5,7.0,1,1461.0,294.0,452.0
4,5.0,6.0,0.00,0.00,0.00,-14.29,NaN,NaN,0.0,0.0,1.0,29.0,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481,223.0,NaN,NaN,NaN,NaN,2.76,26000000.0,26000000.0,2.0,0.0,0.0,17.0,1,1271.0,NaN,NaN
482,222.0,NaN,NaN,NaN,NaN,NaN,NaN,25000000.0,3.0,0.0,0.0,8.0,1,301.0,NaN,NaN
483,221.0,NaN,NaN,NaN,NaN,35.76,70000000.0,81090000.0,3.0,0.0,0.0,6.0,1,1081.0,294.0,1104.0
484,221.0,NaN,NaN,NaN,NaN,41.03,NaN,78320000.0,14.0,0.0,0.0,11.0,1,402.0,NaN,NaN
